In [1]:
# tensorflow 2.0에서는 오류가 발생
# conda activate keras205 
# nohup jupyter notebook --ip 192.168.1.120 & 

import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
sess = tf.Session()
K.set_session(sess)

In [3]:
from keras.layers import Dense, Dropout
from keras.metrics import categorical_accuracy, categorical_crossentropy

In [4]:
class DNN():
    def __init__(self, Nin, Nh_l, Nout):
        self.X_ph = tf.placeholder(tf.float32, shape=(None, Nin))
        self.L_ph = tf.placeholder(tf.float32, shape=(None, Nout))
        
        H = Dense(Nh_l[0], activation='relu')(self.X_ph)
        H = Dropout(0.5)(H)
        H = Dense(Nh_l[1], activation='relu')(H)
        H = Dropout(0.25)(H)
        self.Y_tf = Dense(Nout, activation='softmax')(H)
        self.Loss_tf = tf.reduce_mean(categorical_crossentropy(self.L_ph, self.Y_tf))
        self.Train_tf = tf.train.AdamOptimizer().minimize(self.Loss_tf)
        self.Acc_tf = categorical_accuracy(self.L_ph, self.Y_tf)
        self.Init_tf = tf.global_variables_initializer()

In [5]:
import numpy as np
from keras import datasets
from keras.utils import np_utils

def Data_func():
    (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
    
    Y_train = np_utils.to_categorical(y_train)
    Y_test = np_utils.to_categorical(y_test)
    
    L, W, H = X_train.shape
    print('X_train.shape', X_train.shape)
    X_train = X_train.reshape(-1, W * H)
    print('X_train.shape', X_train.shape)
    X_test = X_test.reshape(-1, W * H)
    
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    
    return (X_train, Y_train), (X_test, Y_test)

In [6]:
from keraspp.skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

In [7]:
def run(model, data, sess, epochs, batch_size=100):
    (X_train, Y_train), (X_test, Y_test) = data
    sess.run(model.Init_tf)
    with sess.as_default():
        N_tr = X_train.shape[0]
        for epoch in range(epochs):
            for b in range(N_tr // batch_size):
                X_tr_b = X_train[batch_size * (b-1):batch_size * b]
                Y_tr_b = Y_train[batch_size * (b-1):batch_size * b]
                
                model.Train_tf.run(feed_dict={model.X_ph: X_tr_b, model.L_ph: Y_tr_b, K.learning_phase(): 1})
                
            loss = sess.run(model.Loss_tf, feed_dict={model.X_ph: X_test, model.L_ph: Y_test, K.learning_phase(): 0})
            acc = model.Acc_tf.eval(feed_dict={model.X_ph: X_test, model.L_ph: Y_test, K.learning_phase(): 0})
            print("Epoch {0}: loss = {1:.3f}, acc = {2:.3f}".format(epoch, loss, np.mean(acc)))

In [8]:
def main():
    Nin = 784
    Nh_l = [100, 50]
    number_of_class = 10
    Nout = number_of_class
    data = Data_func()
    model = DNN(Nin, Nh_l, Nout)
    run(model, data, sess, 10, 100)

In [9]:
main()

X_train.shape (60000, 28, 28)
X_train.shape (60000, 784)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0: loss = 0.223, acc = 0.932
Epoch 1: loss = 0.168, acc = 0.949
Epoch 2: loss = 0.146, acc = 0.956
Epoch 3: loss = 0.135, acc = 0.960
Epoch 4: loss = 0.120, acc = 0.964
Epoch 5: loss = 0.116, acc = 0.967
Epoch 6: loss = 0.113, acc = 0.968
Epoch 7: loss = 0.107, acc = 0.969
Epoch 8: loss = 0.104, acc = 0.968
Epoch 9: loss = 0.103, acc = 0.970


In [10]:
tf.__version__

'1.15.2'